# Exponential Model
This model is not working! It attempts to fit an exponential curve to the data in order to predict the number of new cases. The example used here just takes in one feature and needs to be able to take an `*args` value and unpack it to be able to expand the function to take more arguments. Might be easier/more efficient to just transform the output, make the model and untransform it back. Yeo-johnson or power transform??? just trying to predict `x` for `tomorrow_cases = today_cases**x` instead of predicting `tomorrow cases directly`

In [49]:
import pickle
import os
import urllib.request
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Validation Data?

In [2]:
path_to_ips_file="validation/data/2020-09-30_historical_ip.csv"
input_file = pd.read_csv(path_to_ips_file, low_memory=False)

In [3]:
# input_file[input_file['CountryName'] == 'United States']
input_file[input_file['RegionName'] == 'California']

,CountryName,RegionName,Date,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
50690,United States,California,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50691,United States,California,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50692,United States,California,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50693,United States,California,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50694,United States,California,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50959,United States,California,2020-09-26,3.0,2.0,2.0,4.0,0.0,1.0,1.0,3.0,2.0,3.0,2.0,3.0
50960,United States,California,2020-09-27,3.0,2.0,2.0,4.0,0.0,1.0,1.0,3.0,2.0,3.0,2.0,3.0
50961,United States,California,2020-09-28,3.0,2.0,2.0,4.0,0.0,1.0,1.0,3.0,2.0,3.0,2.0,3.0
50962,United States,California,2020-09-29,3.0,2.0,2.0,4.0,0.0,1.0,1.0,3.0,2.0,3.0,2.0,3.0


## Importing the Training Data

In [9]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local files
data_path = 'examples/predictors/ryan_predictor/data'
DATA_FILE = data_path + '/OxCGRT_latest.csv'


if not os.path.exists(data_path):
    os.mkdir(data_path)
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('examples/predictors/ryan_predictor/data/OxCGRT_latest.csv',
 <http.client.HTTPMessage at 0x7ff72e9461c0>)

In [177]:
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)
# df[cases_df['RegionName'] == 'California']

In [178]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'H7_Vaccination policy', 'H7_Flag',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDispla

In [179]:
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [180]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [181]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [182]:
# import sys
# NewCases = []
# for val in df['NewCases']:
#     if val != 0:
#         NewCases.append(val)
#     else:
#         NewCases.append(sys.float_info.epsilon)
# sys.float_info.epsilon
# df['NewCasesPercent'] = df.groupby('GeoID').NewCases.diff().fillna(0)/NewCases
df['NewCasesPercent'] = df.groupby('GeoID').NewCases.diff().fillna(0)/df['NewCases']
# NewCasesList = df['NewCasesPercent'].tolist()
df = df.replace([np.inf, -np.inf, np.nan], 0)
NewCasesList = df['NewCasesPercent'].tolist()
NewCasesList

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.4,
 0.5454545454545454,
 -1.2,
 0.6153846153846154,
 -2.25,
 0.0,
 1.0,
 0.0,
 1.0,
 -1.5,
 0.0,
 0.0,
 1.0,
 -1.3333333333333333,
 0.0,
 0.4,
 -0.25,
 0.3333333333333333,
 0.0,
 0.0,
 0.0,
 1.0,
 0.5,
 -1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0

In [183]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
#cases_col = ['NewCases', 'NewCasesPercent', 'ConfirmedCases']
cases_col = ['NewCasesPercent']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [184]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCasesPercent.apply(
    lambda group: group.interpolate()).fillna(0))

In [185]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [186]:
df

,CountryName,RegionName,GeoID,Date,NewCasesPercent,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,0,Aruba__nan,2020-01-01,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,0,Aruba__nan,2020-01-02,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,0,Aruba__nan,2020-01-03,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,0,Aruba__nan,2020-01-04,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,0,Aruba__nan,2020-01-05,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92958,Zimbabwe,0,Zimbabwe__nan,2020-07-27,0.593750,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
92959,Zimbabwe,0,Zimbabwe__nan,2020-07-28,-0.699115,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
92960,Zimbabwe,0,Zimbabwe__nan,2020-07-29,-0.822581,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
92961,Zimbabwe,0,Zimbabwe__nan,2020-07-30,0.708920,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0


## Making the Model

In [187]:
# Set number of past days to use to make predictions
nb_lookback_days = 30

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d + 1]
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [188]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [189]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.2,
                                                    random_state=301)

In [190]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
model = Lasso(alpha=0.1,
              precompute=True,
              max_iter=10000,
              positive=True,
              selection='random')
# Fit model
model.fit(X_train, y_train)

Lasso(alpha=0.1, max_iter=10000, positive=True, precompute=True,
      selection='random')

In [191]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 0.4709144568587262
Test MAE: 0.42944358872060706


In [192]:
# Evaluate model
train_preds = model.predict(X_train)
#train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
# y_train
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
#test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 0.5236120633926784
Test MAE: 0.4847246155807235


In [196]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [197]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
# y_train
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 0.4708342501539647
Test MAE: 0.42940659821165933


In [195]:
import numpy as np
from scipy.optimize import curve_fit

def func_exp(x, a, b, c):
        #c = 0
        return a * np.exp(b * x) + c

def exponential_regression (x_data, y_data):
    popt, pcov = curve_fit(func_exp, x_data, y_data, p0 = (-1, 0.01, 1))
    print(popt)
    puntos = plt.plot(x_data, y_data, 'x', color='xkcd:maroon', label = "data")
    curva_regresion = plt.plot(x_data, func_exp(x_data, *popt), color='xkcd:teal', label = "fit: {:.3f}, {:.3f}, {:.3f}".format(*popt))
    return func_exp(x_data, *popt)

# x_data = np.arange(0, 51) 
# y_data = np.array([0.001, 0.199, 0.394, 0.556, 0.797, 0.891, 1.171, 1.128, 1.437, 
#         1.525, 1.720, 1.703, 1.895, 2.003, 2.108, 2.408, 2.424,2.537, 
#         2.647, 2.740, 2.957, 2.58, 3.156, 3.051, 3.043, 3.353, 3.400, 
#         3.606, 3.659, 3.671, 3.750, 3.827, 3.902, 3.976, 4.048, 4.018, 
#         4.286, 4.353, 4.418, 4.382, 4.444, 4.485, 4.465, 4.600, 4.681, 
#         4.737, 4.792, 4.845, 4.909, 4.919, 5.100])
# exponential_regression(x_data, y_data)
exponential_regression(X_train, list(y_train))

ValueError: operands could not be broadcast together with shapes (38729,390) (38729,) 

In [ ]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

In [ ]:
# Save model to file

model_path = 'examples/predictors/ryan_predictor/model'

if not os.path.exists(model_path):
    os.mkdir(model_path)
with open(model_path + '/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluating the Model

In [ ]:
# Reload the module to get the latest changes
from examples.predictors.linear import predict
from importlib import reload
reload(predict)
from examples.predictors.linear.predict import predict_df

In [ ]:
%%time
path_to_ips_file="validation/data/2020-09-30_historical_ip.csv"
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file, verbose=True)

In [ ]:
# Check the predictions
preds_df.head()

## Validation

This is how the predictor is going to be called during the competition.
!!! PLEASE DO NOT CHANGE THE API !!!

In [44]:
!python examples/predictors/linear/ryan_predict.py -s 2020-08-01 -e 2020-08-04 -ip validation/data/2020-09-30_historical_ip.csv -o examples/predictors/ryan_predictor/predictions/2020-08-01_2020-08-04.csv

python: can't open file 'examples/predictors/linear/ryan_predict.py': [Errno 2] No such file or directory


In [45]:
!head predictions/2020-08-01_2020-08-04.csv

head: predictions/2020-08-01_2020-08-04.csv: No such file or directory


## Test Cases
We can generate a prediction file. Let's validate a few cases...

In [46]:
import sys
from validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python examples/predictors/linear/predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

### 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [48]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="validation/data/2020-09-30_historical_ip.csv",
         output_file="examples/predictors/ryan_predictor/predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Traceback (most recent call last):
  File "examples/predictors/linear/predict.py", line 197, in <module>
    predict(args.start_date, args.end_date, args.ip_file, args.output_file)
  File "examples/predictors/linear/predict.py", line 52, in predict
    preds_df = predict_df(start_date, end_date, path_to_ips_file, verbose=False)
  File "examples/predictors/linear/predict.py", line 92, in predict_df
    hist_cases_df = pd.read_csv(DATA_FILE,
  File "/Users/rlew/opt/miniconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 686, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/Users/rlew/opt/miniconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 452, in _read
    parser = TextFileReader(fp_or_buf, **kwds)
  File "/Users/rlew/opt/miniconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 936, in __init__
    self._make_engine(self.engine)
  File "/Users/rlew/opt/miniconda3/lib/python3.8/site-

FileNotFoundError: [Errno 2] No such file or directory: 'examples/predictors/ryan_predictor/predictions/val_4_days.csv'

### 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [ ]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="validation/data/future_ip.csv",
         output_file="examples/predictors/linear/predictions/val_1_month_future.csv")

### 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

In [ ]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

In [ ]:
from validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'examples/predictors/linear/data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "examples/predictors/linear/predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

### Check it

In [ ]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="examples/predictors/linear/predictions/val_6_month_future.csv")